# Kaggle Titanic
[데이터 다운로드](https://www.kaggle.com/competitions/titanic/data)


코드 부분은 지우고 결과 부분만 남겨 뒀습니다.  
남아있는 결과와 동일하게 출력될 수 있도록 코드를 작성해보세요!  

### 사용 라이브러리

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib

### Data Load

In [3]:
train = pd.read_csv("data/train.csv", encoding="utf-8")
train.shape

FileNotFoundError: [Errno 2] No such file or directory: 'data/train.csv'

### 기본 정보

In [ ]:
train.info()

### 결측치 확인

In [ ]:
plt.figure(figsize=(3,3))
sns.heatmap(train.isnull())

In [ ]:
train[["Age","Cabin","Embarked"]].isnull().mean()

### 생존자

In [ ]:
plt.title("Survival")
sns.countplot(x=train["Survived"])

In [ ]:
plt.title("Survival by Sex")
sns.countplot(x=train["Survived"], hue=train["Sex"])

In [ ]:
plt.title("Survival by Pclass")
sns.countplot(x=train["Survived"], hue=train["Pclass"])

### Data Cleaning
#### Age

In [ ]:
sns.boxplot(data=train, x="Pclass", y="Age")

In [ ]:
train.groupby("Pclass")["Age"].describe()

결측치가 있는 `Age`를 `Pclass`의 평균 나이값을 이용해 대채

In [ ]:
age_pclass(1)

In [ ]:
def age_pclass(cols):
    # 직접 작성하시면 됩니다
    return train.groupby("Pclass")["Age"].describe()["mean"][cols]

In [ ]:
# age_pclass 함수를 이용해 Age 결측치 채우기
train.loc[(train["Age"].isnull())&(train['Pclass'] == 1), 'Age'] = age_pclass(1)
train.loc[(train["Age"].isnull())&(train['Pclass'] == 2), 'Age'] = age_pclass(2)
train.loc[(train["Age"].isnull())&(train['Pclass'] == 3), 'Age'] = age_pclass(3)

#### 범주형 데이터 변환

범주형 데이터는 학습에서 바로 사용이 불가능하기 때문에 수치형으로 변환이 필요합니다.  
예를 들어, `Sex`의 경우 `Male=0`, `Female=1`과 같이 변환을 합니다.

In [ ]:
train.info()

현재 범주형 데이터는, `Name, Sex, Ticket, Embarked`가 있음

#### pd.get_dummies
학습 이전에 모든 데이터를 수치로 변환해주는 전처리 작업이 필요함  
1. 수치형 데이터로 변환
2. 수치화된 데이터를 가변수화

##### Note!!! - 가변수화
수치형 데이터로만 변환하면, **서로 간의 관계성**이 생기게 되어버림
- 월, 화, 수 ... -> 1, 2, 3 ...
- 1+2=3 / 월+화=수라는 관계성이 성립해버림

`pandas`의 `get_dummies` 함수는 원핫인코딩을 이용하는 방식  
`get_dummies(data=, prefix=, prefix_sep=, dummy_na=, columns=, drop_first=, dtype=)` 형식으로 존재함
- data: one-hot encoding을 적용할 데이터
- prefix: 생성할 더미 데이터(가변수화된 데이터)의 컬럼 prefix
- prefix_sep: `default='_'`
- dummpy_na: `NaN`도 포함시킬지, `default=False`
- columns: 대상 컬럼
- drop_first: k개의 범주를 k-1개로 가변수화할 경우 사용
- dtype: 새로운 columns의 데이터 타입

In [ ]:
train["male"] = pd.get_dummies(train["Sex"], drop_first=False)["female"]

In [ ]:
train = pd.concat([train, pd.get_dummies(train["Embarked"], drop_first=True)], axis=1)

In [ ]:
train[["PassengerId", "Survived", "Pclass", "Age","SibSp","Parch","Fare","male","Q","S"]].head()